# Notebook Overview
This Notebook takes `../data/03_preprocessed/01_data.csv`, produced by the previous Notebook `00_Process-CARLA-Output.ipynb`, and produces an `Enviornment` object to be used with Trajectron++. The `Environment` object is saved to a pickle file, `../data/03_preprocessed/02_data.pkl`, to be used with the next Notebook, `02_Evaluate-Data.ipynb`.

Much of the code in this Notebook is adapted from code found in Trajectron++'s `Trajectron-plus-plus/experiments/nuScenes/process_data.py` script.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import dill
import argparse
#from tqdm import tqdm
from pyquaternion import Quaternion
#from kalman_filter import NonlinearKinematicBicycle
from sklearn.model_selection import train_test_split

#nu_path = './devkit/python-sdk/'
#sys.path.append(nu_path)
sys.path.append("../../../trajectron")
#from nuscenes.nuscenes import NuScenes
#from nuscenes.map_expansion.map_api import NuScenesMap
#from nuscenes.utils.splits import create_splits_scenes
from environment import Environment, Scene, Node, GeometricMap, derivative_of

from tqdm.notebook import tqdm

# Config Constants

In [ ]:
# Config Constants
FREQUENCY = 3 # this needs to match data sampling frequency
dt = 1 / FREQUENCY

INFILE = '../data/03_preprocessed/01_data.csv'
OUTFILE = '../data/03_preprocessed/02_data.pkl'

# Boiler Plate

In [ ]:
data_columns_vehicle = pd.MultiIndex.from_product([['position', 'velocity', 'acceleration', 'heading'], ['x', 'y']])
data_columns_vehicle = data_columns_vehicle.append(pd.MultiIndex.from_tuples([('heading', '°'), ('heading', 'd°')]))
data_columns_vehicle = data_columns_vehicle.append(pd.MultiIndex.from_product([['velocity', 'acceleration'], ['norm']]))

data_columns_pedestrian = pd.MultiIndex.from_product([['position', 'velocity', 'acceleration'], ['x', 'y']])

standardization = {
    'PEDESTRIAN': {
        'position': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 2},
            'y': {'mean': 0, 'std': 2}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        }
    },
    'VEHICLE': {
        'position': {
            'x': {'mean': 0, 'std': 80},
            'y': {'mean': 0, 'std': 80}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 15},
            'y': {'mean': 0, 'std': 15},
            'norm': {'mean': 0, 'std': 15}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 4},
            'y': {'mean': 0, 'std': 4},
            'norm': {'mean': 0, 'std': 4}
        },
        'heading': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1},
            '°': {'mean': 0, 'std': np.pi},
            'd°': {'mean': 0, 'std': 1}
        }
    }
}

# Initialize Empty Environmnet

In [ ]:
env = Environment(node_type_list=['VEHICLE', 'PEDESTRIAN'], standardization=standardization)

attention_radius = dict()
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 10.0
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.VEHICLE)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.PEDESTRIAN)] = 20.0
attention_radius[(env.NodeType.VEHICLE, env.NodeType.VEHICLE)] = 30.0

env.attention_radius = attention_radius
env.robot_type = None
scenes = []

# Create Trajectron++ DataFrame from Our CSV Format

In [ ]:
df = pd.read_csv(INFILE)

data = pd.DataFrame(columns=['frame_id',
                             'type',
                             'node_id',
                             'robot',
                             'x', 'y', 'z',
                             'length',
                             'width',
                             'height',
                             'heading'])

for i in tqdm(range(df.shape[0])):
    row = df.iloc[i]
    data_point = pd.Series({'frame_id': int(row['Frame']),
                            'type': env.NodeType.VEHICLE,
                            'node_id': int(row['NodeID']),
                            'robot': False,
                            'x': row['Pos_X'],
                            'y': row['Pos_Y'],
                            'z': row['Pos_Z'],
                            'length': 0,
                            'width': 0,
                            'height': 0,
                            'heading': row['Heading']
                           })
    data = data.append(data_point, ignore_index=True)

data.sort_values('frame_id', inplace=True)
max_timesteps = data['frame_id'].max()

# Create Nodes from DataFrame

In [ ]:
scene = Scene(timesteps=max_timesteps + 1, dt=dt, name='default')
scenes.append(scene)

for node_id in pd.unique(data['node_id']):
    node_frequency_multiplier = 1
    node_df = data[data['node_id'] == node_id]

    # If data is prepared correctly, this block should not trigger
    if node_df['x'].shape[0] < 2:
        print('Not enough', node_id)
        continue

    # If data is prepared correctly, this block should not trigger
    if not np.all(np.diff(node_df['frame_id']) == 1):
        print('Occlusion', node_id)
        continue

    node_values = node_df[['x', 'y']].values
    x = node_values[:, 0]
    y = node_values[:, 1]
    heading = node_df['heading'].values

    vx = derivative_of(x, scene.dt)
    vy = derivative_of(y, scene.dt)
    ax = derivative_of(vx, scene.dt)
    ay = derivative_of(vy, scene.dt)

    # Our data should only have VEHICLEs
    if node_df.iloc[0]['type'] == env.NodeType.VEHICLE:
        v = np.stack((vx, vy), axis=-1)
        v_norm = np.linalg.norm(np.stack((vx, vy), axis=-1), axis=-1, keepdims=True)
        heading_v = np.divide(v, v_norm, out=np.zeros_like(v), where=(v_norm > 1.))
        heading_x = heading_v[:, 0]
        heading_y = heading_v[:, 1]

        data_dict = {('position', 'x'): x,
                     ('position', 'y'): y,
                     ('velocity', 'x'): vx,
                     ('velocity', 'y'): vy,
                     ('velocity', 'norm'): np.linalg.norm(np.stack((vx, vy), axis=-1), axis=-1),
                     ('acceleration', 'x'): ax,
                     ('acceleration', 'y'): ay,
                     ('acceleration', 'norm'): np.linalg.norm(np.stack((ax, ay), axis=-1), axis=-1),
                     ('heading', 'x'): heading_x,
                     ('heading', 'y'): heading_y,
                     ('heading', '°'): heading,
                     ('heading', 'd°'): derivative_of(heading, dt, radian=True)}
        node_data = pd.DataFrame(data_dict, columns=data_columns_vehicle)
    else:
        data_dict = {('position', 'x'): x,
                     ('position', 'y'): y,
                     ('velocity', 'x'): vx,
                     ('velocity', 'y'): vy,
                     ('acceleration', 'x'): ax,
                     ('acceleration', 'y'): ay}
        node_data = pd.DataFrame(data_dict, columns=data_columns_pedestrian)

    node = Node(node_type=node_df.iloc[0]['type'], node_id=str(node_id), data=node_data, frequency_multiplier=node_frequency_multiplier)
    node.first_timestep = node_df['frame_id'].iloc[0]

    scene.nodes.append(node)

# Write Environment as Pickle

In [ ]:
env.scenes = scenes

data_dict_path = os.path.join(OUTFILE)
with open(data_dict_path, 'wb') as f:
    dill.dump(env, f, protocol=dill.HIGHEST_PROTOCOL)
print('Saved Environment!')